In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as se
import time
import timeit
%matplotlib inline
import datetime

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [26]:
dataset = pd.read_csv('/content/gdrive/MyDrive/CausaLens/AirQualityUCI/AirQualityUCI.csv', sep = ';', decimal=',')
dataset.head()

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Unnamed: 15,Unnamed: 16
0,10/03/2004,18.00.00,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578,NaN,NaN
1,10/03/2004,19.00.00,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255,NaN,NaN
2,10/03/2004,20.00.00,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502,NaN,NaN
3,10/03/2004,21.00.00,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867,NaN,NaN
4,10/03/2004,22.00.00,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888,NaN,NaN


In [27]:
dataset['target']=dataset['CO(GT)'].shift(1)
dataset.head()

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Unnamed: 15,Unnamed: 16,target
0,10/03/2004,18.00.00,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578,NaN,NaN,NaN
1,10/03/2004,19.00.00,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255,NaN,NaN,2.6
2,10/03/2004,20.00.00,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502,NaN,NaN,2.0
3,10/03/2004,21.00.00,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867,NaN,NaN,2.2
4,10/03/2004,22.00.00,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888,NaN,NaN,2.2


In [32]:
dataset.isna().sum()

Date                0
Time                0
CO(GT)              0
PT08.S1(CO)         0
NMHC(GT)            0
C6H6(GT)            0
PT08.S2(NMHC)       0
NOx(GT)             0
PT08.S3(NOx)        0
NO2(GT)             0
PT08.S4(NO2)        0
PT08.S5(O3)         0
T                   0
RH                  0
AH                  0
Unnamed: 15      9356
Unnamed: 16      9356
target              0
dtype: int64

In [31]:
dataset=dataset.dropna(subset=['Time'], inplace=False)
dataset.head()

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Unnamed: 15,Unnamed: 16,target
1,10/03/2004,19.00.00,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255,NaN,NaN,2.6
2,10/03/2004,20.00.00,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502,NaN,NaN,2.0
3,10/03/2004,21.00.00,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867,NaN,NaN,2.2
4,10/03/2004,22.00.00,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888,NaN,NaN,2.2
5,10/03/2004,23.00.00,1.2,1197.0,38.0,4.7,750.0,89.0,1337.0,96.0,1393.0,949.0,11.2,59.2,0.7848,NaN,NaN,1.6


In [33]:
New=dataset.drop(columns=['Date','Time','Unnamed: 15','Unnamed: 16'])
New.head()

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,target
1,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255,2.6
2,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502,2.0
3,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867,2.2
4,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888,2.2
5,1.2,1197.0,38.0,4.7,750.0,89.0,1337.0,96.0,1393.0,949.0,11.2,59.2,0.7848,1.6


In [34]:
New.corr(method ='pearson') 

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,target
CO(GT),1.000000,0.041366,0.128274,-0.031391,0.029904,0.526458,-0.090026,0.671120,-0.073759,0.080280,-0.068944,-0.048237,-0.045903,0.865878
PT08.S1(CO),0.041366,1.000000,0.169837,0.852706,0.933111,0.278007,0.086945,0.153995,0.845143,0.892432,0.754871,0.745394,0.764921,0.037374
NMHC(GT),0.128274,0.169837,1.000000,0.037275,0.110029,-0.004426,0.048644,0.103233,0.162574,0.101062,-0.000030,0.008243,0.012457,0.123690
C6H6(GT),-0.031391,0.852706,0.037275,1.000000,0.767432,-0.001174,0.512191,-0.011004,0.774676,0.641334,0.971376,0.925062,0.984555,-0.031283
PT08.S2(NMHC),0.029904,0.933111,0.110029,0.767432,1.000000,0.331276,-0.073709,0.176471,0.874780,0.909904,0.669028,0.585801,0.646571,0.024740
NOx(GT),0.526458,0.278007,-0.004426,-0.001174,0.331276,1.000000,-0.436099,0.817148,0.035547,0.461900,-0.138452,-0.053009,-0.095847,0.508300
PT08.S3(NOx),-0.090026,0.086945,0.048644,0.512191,-0.073709,-0.436099,1.000000,-0.256281,0.122685,-0.208932,0.588124,0.573554,0.621625,-0.084541
NO2(GT),0.671120,0.153995,0.103233,-0.011004,0.176471,0.817148,-0.256281,1.000000,-0.022204,0.253418,-0.084109,-0.081314,-0.060449,0.651210
PT08.S4(NO2),-0.073759,0.845143,0.162574,0.774676,0.874780,0.035547,0.122685,-0.022204,1.000000,0.723678,0.755071,0.640709,0.691916,-0.076524
PT08.S5(O3),0.080280,0.892432,0.101062,0.641334,0.909904,0.461900,-0.208932,0.253418,0.723678,1.000000,0.503705,0.524955,0.519466,0.075966


In [22]:
#correlation = New.target.corr(New.RH)
#New=pd.to_numeric(New.AH)
New.convert_dtypes().dtypes  

CO(GT)           string
PT08.S1(CO)       Int64
NMHC(GT)          Int64
C6H6(GT)         string
PT08.S2(NMHC)     Int64
NOx(GT)           Int64
PT08.S3(NOx)      Int64
NO2(GT)           Int64
PT08.S4(NO2)      Int64
PT08.S5(O3)       Int64
T                string
RH               string
AH               string
target           string
dtype: object

In [25]:
dataset.AH

1       0,7255
2       0,7502
3       0,7867
4       0,7888
5       0,7848
         ...  
9352    0,7568
9353    0,7119
9354    0,6406
9355    0,5139
9356    0,5028
Name: AH, Length: 9356, dtype: object